use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle

In [2]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 10000
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id].to('cuda:7')
            product_emb = product_embeddings[batch_product_id].to('cuda:7')
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge valid score

In [6]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [7]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [8]:
FIELD_NAME = 'text_bert_scores'

In [9]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_results_DE/results/item_reps/reordered_item.npy'
DE_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_results_DE/valid_results/valid_query_reps/query.npy'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_results_JP/results/item_reps/reordered_item.npy'
JP_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_results_JP/valid_results/valid_query_reps/query.npy'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_roberta_results_UK/results/item_reps/reordered_item.npy'
UK_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_roberta_results_UK/valid_results/valid_query_reps/query.npy'

In [10]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/b5aeac4e5b9ff0518bbcb59a28086594'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/2536617955df215e0047f5b220d1c012'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/8d133ea55ad67bd3efd625dfeff0fb1d'

In [11]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [12]:
merged_candidates = read_merged_candidates_feature()
valid_sessions = read_valid_sessions()
EMBED_DIM = 768
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [13]:
# sess embeddings 
valid_DE_query_emb = torch.from_numpy(np.load(DE_valid_embeddings_path)) 
valid_JP_query_emb = torch.from_numpy(np.load(JP_valid_embeddings_path))
valid_UK_query_emb = torch.from_numpy(np.load(UK_valid_embeddings_path))
valid_query_embeddings = torch.empty(len(valid_sessions), EMBED_DIM)
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'DE'].index).tolist()] = valid_DE_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'JP'].index).tolist()] = valid_JP_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'UK'].index).tolist()] = valid_UK_query_emb

In [14]:
valid_query_embeddings, valid_query_embeddings.shape

(tensor([[-0.1100,  0.3310,  0.0674,  ...,  0.1768,  0.2052, -0.3436],
         [ 0.2061,  0.5285,  0.0776,  ..., -0.0646,  0.1119, -0.1762],
         [-0.0226,  0.1935, -0.0137,  ...,  0.0779, -0.0126, -0.4134],
         ...,
         [-0.0344,  0.0680, -0.0033,  ..., -0.4798,  0.0155, -0.1162],
         [ 0.0743,  0.1279, -0.0812,  ..., -0.2926,  0.0121,  0.0147],
         [ 0.1461,  0.0491, -0.1343,  ..., -0.1492, -0.0622,  0.0735]]),
 torch.Size([261816, 768]))

In [15]:
# product_embeddings, the embeddings include padding embedding
DE_product_emb = torch.from_numpy(np.load(DE_product_embeddings_path)).type(torch.float)
JP_product_emb = torch.from_numpy(np.load(JP_product_embeddings_path)).type(torch.float)
UK_product_emb = torch.from_numpy(np.load(UK_product_embeddings_path)).type(torch.float)
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [16]:
DE_product_emb

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2478,  0.4152,  0.3816,  ...,  0.4283, -0.3134, -0.0649],
        [ 0.1436,  0.0728,  0.3197,  ...,  0.0514,  0.2001, -0.2141],
        ...,
        [ 0.0069,  0.4923, -0.1382,  ...,  0.2253,  0.4376, -0.3512],
        [-0.3916, -0.1836,  0.3598,  ...,  0.0748, -0.3209, -0.1415],
        [-0.2131,  0.5381, -0.2594,  ...,  0.3637, -0.2117, -0.1636]])

In [17]:
merged_candidates_ = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [18]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [19]:
# merged_candidates_g = cudf.from_pandas(merged_candidates_)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [20]:
# merged_candidates_score_g = merged_candidates_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_score_g['dataset_id'] = merged_candidates_score_g['dataset_id'].fillna(0)
# merged_candidates_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_score_g = merged_candidates_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_score = merged_candidates_score_g.to_pandas()

In [21]:
merged_candidates_score = merged_candidates_.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_score['dataset_id'] = merged_candidates_score['dataset_id'].fillna(0)
merged_candidates_score.drop(columns=['locale'], inplace=True)
merged_candidates_score = merged_candidates_score.sort_values(by=['sess_id', 'product'])
merged_candidates_score.reset_index(drop=True, inplace=True)
assert len(merged_candidates_score) == len(merged_candidates)

In [22]:
# del merged_candidates_g
# del product_id_df_g
# del merged_candidates_score_g

In [23]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \
        merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_165895/2978421209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \


In [24]:
# using cosine scores 
normalized_valid_query_embeddings = torch.nn.functional.normalize(valid_query_embeddings, p=2, dim=-1)
normalized_product_embeddings = torch.nn.functional.normalize(product_embeddings, p=2, dim=-1)
normalized_product_embeddings = normalized_product_embeddings.type(torch.float)

In [25]:
# cosine scores
merged_candidates_score['cos_'+FIELD_NAME] = get_scores(merged_candidates_score, normalized_valid_query_embeddings, normalized_product_embeddings)

  0%|                                                                                                                  | 0/7885 [00:00<?, ?it/s]/tmp/ipykernel_165895/3457709309.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 7885/7885 [08:32<00:00, 15.37it/s]


In [26]:
merged_candidates_score[FIELD_NAME] = get_scores(merged_candidates_score, valid_query_embeddings, product_embeddings)

  0%|                                                                                                                                   | 0/7885 [00:00<?, ?it/s]/tmp/ipykernel_165895/3457709309.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7885/7885 [09:17<00:00, 14.14it/s]


In [27]:
normalize_scores(merged_candidates_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [28]:
merged_candidates['cos_'+FIELD_NAME] = merged_candidates_score['cos_'+FIELD_NAME]

In [29]:
merged_candidates[FIELD_NAME] = merged_candidates_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_score['normalized_'+FIELD_NAME]

In [30]:
cast_dtype(merged_candidates, ['cos_'+FIELD_NAME])

In [31]:
cast_dtype(merged_candidates, [FIELD_NAME, 'normalized_'+FIELD_NAME])
merged_candidates.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [32]:
# verify gru4rec scores
merged_candidates[merged_candidates['sess_id'] == 150009].sort_values(by=['sasrec_scores_2'], ascending=False)[['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2', 'normalized_'+FIELD_NAME, FIELD_NAME, 'cos_'+FIELD_NAME]].iloc[:25]

,sess_locale,product,normalized_sasrec_scores_2,sasrec_scores_2,normalized_text_bert_scores,text_bert_scores,cos_text_bert_scores
45145287,JP,B0BB78W1K4,0.109824,12.599373,0.161231,196.879623,0.949360
45145274,JP,B0B7LFKRBW,0.085752,12.351955,0.010322,194.131088,0.936323
45145171,JP,B09FLRJDZV,0.066722,12.101032,0.031126,195.234833,0.929352
45145217,JP,B09VK2DCW4,0.058213,11.964600,0.010719,194.168808,0.936697
45145300,JP,B0BHSYHDC7,0.046944,11.749451,0.103612,196.437439,0.939319
45145098,JP,B0864F9F3X,0.038047,11.539320,0.003106,192.930084,0.932668
45145220,JP,B09VXHMB4B,0.035546,11.471312,0.035155,195.356567,0.940585
45145304,JP,B0BHYZTML3,0.034852,11.451614,0.001140,191.927689,0.922487
45145289,JP,B0BBFYDRPV,0.028480,11.249701,0.011010,194.195557,0.932041
45145230,JP,B09YCVY6PY,0.028295,11.243197,0.000220,190.281631,0.911501


In [33]:
merged_candidates

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,...,all_items_co_graph_count_2,co_graph_counts_0,normalized_co_graph_counts_0,co_graph_counts_1,normalized_co_graph_counts_1,co_graph_counts_2,normalized_co_graph_counts_2,cos_text_bert_scores,text_bert_scores,normalized_text_bert_scores
0,0,DE,355165591X,0.0,43.256542,8.990000,51.0,2.230508,7.658405e-09,0.512931,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.903757,378.286041,1.296655e-08
1,0,DE,3833237058,0.0,43.256542,22.000000,84.0,9.605231,1.221631e-05,9.325538,...,0,1.0,0.002217,0.090909,0.00083,0.0,0.000000,0.921604,387.624756,1.474268e-04
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,7.0,0.714114,1.681035e-09,-0.115904,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.901061,374.802551,3.980740e-10
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,166.0,8.750996,5.199363e-06,8.507557,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.927298,385.701782,2.154962e-05
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,502.0,8.056712,2.596729e-06,5.898870,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.930655,385.398499,1.591202e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,7.0,6.813615,1.076201e-03,7.203015,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.972680,438.956238,1.636532e-04
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,53.0,9.030836,9.881445e-03,10.123234,...,2,14.0,0.017115,4.250000,0.01369,2.0,0.010638,0.972680,438.956238,1.636532e-04
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,7.0,0.796892,2.623396e-06,1.711608,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.953467,430.164368,2.486932e-08
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,38.0,4.250781,8.296004e-05,6.447586,...,0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.961829,434.029083,1.186011e-06
